In [1]:
from pathlib import Path
import os
from IPython.display import HTML
import wandb
from mp_transformer.models.transformer import MovementPrimitiveTransformer
from mp_transformer.config import CONFIG
from mp_transformer.train import setup
from mp_transformer.utils import save_side_by_side_video

In [2]:
current_dir = Path.cwd().parts[-1]
if current_dir == "demo":
    os.chdir("..")
!pwd

/data/daniel/git/mp-transformer


In [51]:
run = wandb.init(project="mp-transformer")
artifact = run.use_artifact("tcs-mr/mp-transformer/model:v22", type='model')
artifact_dir = artifact.download()


wandb:   1 of 1 files downloaded.  


In [44]:
# print(artifact_dir)

In [52]:
CONFIG["hidden_dim"] = 32
CONFIG["latent_dim"] = 16
model, train_dataset, val_dataset = setup(CONFIG)
artifact_dir = "./artifacts/model:v22"
model = model.load_from_checkpoint(Path(artifact_dir, "model.ckpt"), config=CONFIG)
run.finish()

In [53]:
item = val_dataset[-1]
# item = val_dataset[64]
# item = val_dataset[0]
# save_side_by_side_video(item, model, from_idx=3)
save_side_by_side_video(item, model)

Video saved to tmp/comp_vid.mp4


In [39]:

HTML("""
<video width="320" height="240" controls>
  <source src="../tmp/comp_vid.mp4" type="video/mp4">
</video>
""")

In [54]:
save_side_by_side_video(item, model, from_idx=0, path="tmp/gen_vid.mp4")

poses range: [0.008441172540187836, 0.9904189705848694]
mus range: [-1.6846040487289429, 1.0358797311782837]
average mu: -0.03345848619937897
logvars range: [-8.154383659362793, 0.5709149241447449]
median logvar: -3.0112290382385254
gt_latents range: [-2.318274974822998, 2.3208553791046143]
average gt_latents: -0.05669620633125305
random_latents range: [-2.8157382011413574, 2.888730049133301]
average random_latents: -0.054474230855703354
Video saved to tmp/gen_vid.mp4


In [55]:


HTML("""
<video width="320" height="240" controls>
  <source src="../tmp/gen_vid.mp4" type="video/mp4">
</video>
""")